In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
application_df.drop(columns = ['EIN','NAME'],axis=1, inplace=True)

In [ ]:
application_df['ASK_AMT'].dtype

dtype('int64')

In [ ]:
consideration_counts = application_df['SPECIAL_CONSIDERATIONS'].value_counts()
consideration_counts

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

In [ ]:
affiliation_counts = application_df['AFFILIATION'].value_counts()
affiliation_counts

AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of affiliations to be replaced
affiliations_to_replace = affiliation_counts[affiliation_counts < 100].index.tolist()

# Replace in dataframe
for aff in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff,"Other")

In [ ]:
# Check to make sure replacement was successful
application_df['AFFILIATION'].value_counts()

AFFILIATION
Independent         18480
CompanySponsored    15705
Other                 114
Name: count, dtype: int64

In [ ]:
use_counts = application_df['USE_CASE'].value_counts()
use_counts

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of use_cases to be replaced

uses_to_replace = use_counts[use_counts < 500].index.tolist()

# Replace in dataframe
for use in uses_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(use,"Other")

In [ ]:
# Check to make sure replacement was successful
application_df['USE_CASE'].value_counts()

USE_CASE
Preservation    28095
ProductDev       5671
Other             533
Name: count, dtype: int64

In [ ]:
organization_counts = application_df['ORGANIZATION'].value_counts()
organization_counts

ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: count, dtype: int64

leave Organization - making Co-operative and Corporation into other reduces only by one feature

In [ ]:
df_no_special_applications = application_df.loc[application_df['SPECIAL_CONSIDERATIONS']== 'N',:]

In [ ]:
new_consideration_counts = df_no_special_applications['SPECIAL_CONSIDERATIONS'].value_counts()
new_consideration_counts

SPECIAL_CONSIDERATIONS
N    34272
Name: count, dtype: int64

In [ ]:
application_types_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_types_counts[application_types_counts < 600].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [ ]:
classification_types_counts = df_no_special_applications['CLASSIFICATION'].value_counts()
classification_types_counts

CLASSIFICATION
C1000    17309
C2000     6070
C1200     4833
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`


classifications_to_replace = classification_types_counts[classification_types_counts < 2000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    df_no_special_applications['CLASSIFICATION'] = df_no_special_applications['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
df_no_special_applications['CLASSIFICATION'].value_counts()

<ipython-input-20-bf9e9950d152>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_special_applications['CLASSIFICATION'] = df_no_special_applications['CLASSIFICATION'].replace(cls,"Other")


CLASSIFICATION
C1000    17309
C2000     6070
Other     6060
C1200     4833
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_cols = ['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
application_dummies = pd.get_dummies(df_no_special_applications,columns = dummy_cols, dtype=int)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
0,1,5000,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,108590,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,1,5000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_dummies['IS_SUCCESSFUL']
X = application_dummies.drop('IS_SUCCESSFUL', axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1, stratify = y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
# Import the kerastuner library
from tensorflow import keras

!pip install -q -U keras-tuner

import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 25s]
val_accuracy: 0.7257236242294312

Best val_accuracy So Far: 0.7270074486732483
Total elapsed time: 00h 24m 19s


In [ ]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 21,
 'num_layers': 5,
 'units_0': 11,
 'units_1': 26,
 'units_2': 26,
 'units_3': 21,
 'units_4': 6,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [ ]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5555 - accuracy: 0.7302 - 1s/epoch - 5ms/step
Loss: 0.5555050373077393, Accuracy: 0.7301587462425232


In [ ]:
df_no_special_applications.drop(columns=['SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], axis=1, inplace=True)
df_no_special_applications.head()

KeyError: "['SPECIAL_CONSIDERATIONS', 'INCOME_AMT'] not found in axis"

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_cols = ['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION']
application_dummies = pd.get_dummies(df_no_special_applications,columns = dummy_cols, dtype=int)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_dummies['IS_SUCCESSFUL']
X = application_dummies.drop('IS_SUCCESSFUL', axis=1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1, stratify = y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=41))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 00m 49s]
val_accuracy: 0.7223389148712158

Best val_accuracy So Far: 0.7272409200668335
Total elapsed time: 00h 24m 49s
